In [11]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [12]:
class Model1 (nn.Module):
    def __init__(self):
        super(Model1,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(5,3),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(3,1),
            nn.Tanh()
        )
    def forward(self,x):
        return self.model(x)

In [13]:
df = pd.read_json("data2/normal_vpm.json",orient="records",lines=True)


In [14]:
list_data = df.to_numpy()
np.random.shuffle(list_data)

train_x = torch.Tensor(list_data[:10000,:-1])
train_Y = torch.Tensor(list_data[:10000,-1].reshape(-1,1))
train_data = TensorDataset(train_x,train_Y)
train_dataLoader = DataLoader(train_data,batch_size=128,shuffle=True)

test_x = torch.Tensor(list_data[10000:,:-1])
test_Y = torch.Tensor(list_data[10000:,-1].reshape(-1,1))
test_dataset = TensorDataset(test_x,test_Y)
test_dataLoader = DataLoader(test_dataset,batch_size=128,shuffle=True)


In [15]:
test_x[0:10]

tensor([[9.2330e-01, 5.0000e+00, 4.2146e-05, 2.0000e+01, 2.0000e+01],
        [4.7296e-01, 5.0000e+00, 8.2275e-05, 2.0000e+01, 2.0000e+01],
        [5.2098e-01, 5.0000e+00, 7.4692e-05, 2.0000e+01, 2.0000e+01],
        [5.8874e-01, 4.0000e+00, 5.2876e-05, 1.5000e+01, 0.0000e+00],
        [2.7239e-05, 2.0000e+00, 5.5814e-01, 1.0000e+01, 1.0000e+01],
        [3.5564e-01, 3.0000e+00, 6.5650e-05, 1.5000e+01, 1.5000e+01],
        [5.1332e-01, 2.0000e+00, 3.0323e-05, 2.0000e+01, 0.0000e+00],
        [5.9059e-01, 4.0000e+00, 5.2711e-05, 2.0000e+01, 2.0000e+01],
        [2.8277e-05, 2.0000e+00, 5.3812e-01, 1.5000e+01, 1.5000e+01],
        [8.4582e-01, 3.0000e+00, 2.7604e-05, 1.5000e+01, 0.0000e+00]])

In [16]:
model1 = Model1()
optimizer = optim.Adam(model1.parameters(),lr=0.06)
criterion = nn.MSELoss()

In [17]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model1.to(device)
print("Running model on:",device)

Running model on: cpu


In [18]:
epochs = 20

model1.train()
for epoch in tqdm(range(epochs)):
    train_loss = []
    for x, Y in train_dataLoader:
        x = x.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        out = model1(x)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    print(f"Loss: {np.average(train_loss):.3f}")

 10%|█         | 2/20 [00:00<00:01, 15.20it/s]Loss: 0.482
Loss: 0.245
 20%|██        | 4/20 [00:00<00:01, 15.33it/s]Loss: 0.266
Loss: 0.245
Loss: 0.249
 30%|███       | 6/20 [00:00<00:00, 14.94it/s]Loss: 0.243
Loss: 0.234
 50%|█████     | 10/20 [00:00<00:00, 14.24it/s]Loss: 0.243
Loss: 0.227
Loss: 0.230
 60%|██████    | 12/20 [00:00<00:00, 14.18it/s]Loss: 0.226
Loss: 0.230
 80%|████████  | 16/20 [00:01<00:00, 13.87it/s]Loss: 0.235
Loss: 0.231
Loss: 0.231
 90%|█████████ | 18/20 [00:01<00:00, 13.94it/s]Loss: 0.224
Loss: 0.232
100%|██████████| 20/20 [00:01<00:00, 14.24it/s]Loss: 0.228



In [19]:
correct = 0
for x, Y in test_dataLoader:
    model1.eval()
    x = x.to(device)
    Y = Y.cpu()
    out=model1(x)
    out.cpu()
    for i,o in enumerate(out):
        if o * Y[i] > 0:
            correct += 1
print(f'Corectly predicted: {correct} out of {len(test_dataset)}\nTest accuracy: {correct/len(test_dataset):.2%}')


Corectly predicted: 2525 out of 2752
Test accuracy: 91.75%


In [20]:
model1.cpu()
print(model1(torch.Tensor(np.array([[0.5,4,0.00003,5,0],[0.5,4,0.00003,5,5],[0.5,4,0.00003,5,10],[0.5,4,0.00003,5,15],[0.5,4,0.00003,5,20]]))))
#print(model1(torch.Tensor(np.array([60000000,2,0,0]))))

tensor([[-0.9845],
        [-0.4066],
        [-0.0899],
        [ 0.2461],
        [ 0.5318]], grad_fn=<TanhBackward>)
